In [53]:
# LIBRERIAS 
from sklearn.datasets import fetch_california_housing
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from mlxtend.feature_selection import ExhaustiveFeatureSelector as EFS
import numpy as np
import pandas as pd
import math 
import random as rd
inv = np.linalg.inv
import random as rd
import math
log = math.log

In [54]:
# PERMUTACION , obtenemos el subconjunto de tamño n_sub de un total de 8 columnas(en este caso)
# teoria cortesia del libro de simulacion


def permutacion(l,n_sub):
    f = []
    k = len(l)
    while k>1:
        i = math.floor(k*rd.uniform(0,1))
        #print(f"i : {i}")
        l[k-1] , l[i] = l[i] , l[k-1]
        f.append(l[k-1])
        if (len(l) - k) + 1  == n_sub:
            break
        k = k - 1
    #print(f)
    return f
    
if __name__=='__main__':
    l = [0,1,2,3,4,5,6,7]
    p = permutacion(l,5)
    print(p)

[5, 4, 0, 2, 3]


In [25]:
# NORMALIZACION de la data (x - media) /desviacion estandar
# Fase 1 subir y normalizar
def normalizacion(X):
    n , m = X.shape
    medias = np.zeros(m)
    varianzas = np.zeros(m)
    for i in range(m):
        medias[i] = X[:,i].mean()
        varianzas[i] = X[:,i].std()
    for i in range(m):
        X[:,i] = (X[:,i] - medias[i])/varianzas[i]
    return X
    
if __name__=='__main__':
    datos = fetch_california_housing()
    Xe = datos.data
    y = datos.target
    X = normalizacion(Xe)
    print(f"x:\n{X}")

x:
[[ 2.34476576  0.98214266  0.62855945 ... -0.04959654  1.05254828
  -1.32783522]
 [ 2.33223796 -0.60701891  0.32704136 ... -0.09251223  1.04318455
  -1.32284391]
 [ 1.7826994   1.85618152  1.15562047 ... -0.02584253  1.03850269
  -1.33282653]
 ...
 [-1.14259331 -0.92485123 -0.09031802 ... -0.0717345   1.77823747
  -0.8237132 ]
 [-1.05458292 -0.84539315 -0.04021111 ... -0.09122515  1.77823747
  -0.87362627]
 [-0.78012947 -1.00430931 -0.07044252 ... -0.04368215  1.75014627
  -0.83369581]]


In [26]:
# agregar la columna de 1's para la operacion algebraica de modo quee 1*W0  + X*W1 = y_
def X_1(Xe,columna):
    n, m = Xe.shape
    X = np.ones((n,len(columna) + 1))
    for i in range(len(columna)):
        X[:,i + 1] = Xe[:,columna[i]]
    return X

if __name__=='__main__':
    datos = fetch_california_housing()
    Xe = datos.data # con 8 predictores osea 8 vectores columnas
    y = datos.target
    X = normalizacion(Xe)
    columna = [0,2,3] # los indices de las columnas
    X = X_1(X,columna)
    print(f"X:\n{X}")

X:
[[ 1.          2.34476576  0.62855945 -0.15375759]
 [ 1.          2.33223796  0.32704136 -0.26333577]
 [ 1.          1.7826994   1.15562047 -0.04901636]
 ...
 [ 1.         -1.14259331 -0.09031802  0.04941393]
 [ 1.         -1.05458292 -0.04021111  0.15877763]
 [ 1.         -0.78012947 -0.07044252  0.1384028 ]]


In [20]:
#    0   1   2   3   4  , totalmente propio
def subconjuntos_no(lista):
    n = len(lista)
    combinaciones = []
    for j in range(n):
        combinaciones.append([j])
        for grupos in range(2,n + 1):
            #sub.append(j)# sub(j)
            for k in range(n-1,j,-1):
                sub = []
                sub.append(j)
                # sub(k)
                # grupos:2 j,k     sub(k) 0,4   0,3  0,2   0,1   0,0
                # grupos:3  j:0 k:4 
                if k - j + 1 >= grupos:
                    for i in range(0,grupos-1):
                        if (k - i) != j:
                            sub.append(k - i)
                    combinaciones.append(sub)
                    #  j,k    sub (k - i)
                # grupos:2 sub(k) 0,4   0,3  0,2   0,1   0,0
        # grupos:2  0,4   0,3  0,2   0,1   0,0
    return combinaciones

if __name__=='__main__':
    lista = [ 0 ,  1 ,  2 ,  3  , 4 ]
    combinaciones = subconjuntos_no(lista)
    print(f"combinaciones:\n{combinaciones}")    

combinaciones:
[[0], [0, 4], [0, 3], [0, 2], [0, 1], [0, 4, 3], [0, 3, 2], [0, 2, 1], [0, 4, 3, 2], [0, 3, 2, 1], [0, 4, 3, 2, 1], [1], [1, 4], [1, 3], [1, 2], [1, 4, 3], [1, 3, 2], [1, 4, 3, 2], [2], [2, 4], [2, 3], [2, 4, 3], [3], [3, 4], [4]]


In [ ]:
# pero faltan algunas permutaciones
# sugerencia de gpt para el bucle interno
#    0   1   2   3   4 
def subconjuntos_noo(lista):
    n = len(lista)
    combinaciones = []
    for j in range(n):
        combinaciones.append([j])
        for grupos in range(2,n + 1):
            #sub.append(j)# sub(j)
            for k in range(n-1,j,-1):
                sub = []
                sub.append(j)
                # sub(k)
                # grupos:2 j,k     sub(k) 0,4   0,3  0,2   0,1   0,0
                # grupos:3  j:0 k:4 
                if k - j + 1 >= grupos:
                    for salto in range(1, k - j + 1):
                        sub = [j]
                        for i in range(grupos-1):
                            idx = k - i * salto
                            if idx > j:
                                sub.append(idx)
                        if len(sub) == grupos:
                            if sub not in combinaciones:
                                combinaciones.append(sub)
                    #  j,k    sub (k

In [27]:
#GPT completamente, subconjuntos de todos los tamaños , no importa el orden en total 2**n  -1 grupos
def subconjuntos(lista):
    n = len(lista)
    combinaciones = []
    # tamaños de los grupos
    for grupos in range(1, n + 1):
        # índices iniciales: [0,1,2,...,grupos-1]
        indices = list(range(grupos))
        while True:
            # construir combinación actual
            combinaciones.append([lista[i] for i in indices])
            # buscar posición para avanzar
            for i in range(grupos - 1, -1, -1):
                if indices[i] != i + n - grupos:
                    break
            else:
                # no se puede avanzar más
                break
            # avanzar este índice
            indices[i] += 1
            # reajustar los que están a la derecha
            for j in range(i + 1, grupos):
                indices[j] = indices[j - 1] + 1
    return combinaciones

if __name__ == "__main__":
    lista = [0, 1, 2, 3, 4]
    combinaciones = subconjuntos(lista)
    print("Combinaciones:")
    for c in combinaciones:
        print(c)
    print("\nTotal:", len(combinaciones))
# y obtiene todas las combinaciones, gpt es mucho mejor programador que yo, triste pero cierto<s

Combinaciones:
[0]
[1]
[2]
[3]
[4]
[0, 1]
[0, 2]
[0, 3]
[0, 4]
[1, 2]
[1, 3]
[1, 4]
[2, 3]
[2, 4]
[3, 4]
[0, 1, 2]
[0, 1, 3]
[0, 1, 4]
[0, 2, 3]
[0, 2, 4]
[0, 3, 4]
[1, 2, 3]
[1, 2, 4]
[1, 3, 4]
[2, 3, 4]
[0, 1, 2, 3]
[0, 1, 2, 4]
[0, 1, 3, 4]
[0, 2, 3, 4]
[1, 2, 3, 4]
[0, 1, 2, 3, 4]

Total: 31


In [18]:
# NO SE USA
def columnas(subconjunto, indices):
    columna = []
    for e in subconjunto:
        columna.append(indices[e])
    return columna
    
if __name__=='__main__': 
    p = permutacion([0, 1, 2, 3, 4])
    print(f"permutacion:\n{p}")
    subcon = subconjuntos(p)
    print(f"subcon:\n{subcon}")


i : 2
i : 1
i : 2
i : 1
permutacion:
[2, 1, 4, 3]
subcon:
[[2], [1], [4], [3], [2, 1], [2, 4], [2, 3], [1, 4], [1, 3], [4, 3], [2, 1, 4], [2, 1, 3], [2, 4, 3], [1, 4, 3], [2, 1, 4, 3]]


In [41]:
# REGRESION multivariada
def regresion(Xe,y):
    #n , m = Xe.shape
    #X = copiar_columnas(Xe,columna)  # debe ser una dependencia y no estar acoplado 
    theta = inv(Xe.T @ Xe) @ (Xe.T @ y)
    #print(f"theta:\n{theta}")
    return theta

if __name__=='__main__':
    datos = fetch_california_housing()
    X, y = datos.data, datos.target
    X = normalizacion(X)
    X = X_1(X,[4,2,1])
    theta = regresion(X,y)
    print(f"theta:\n{theta} \n para {X}")

theta:
[2.06855817 0.03455457 0.20284921 0.16321063] 
 para [[ 1.         -0.9744286   0.62855945  0.98214266]
 [ 1.          0.86143887  0.32704136 -0.60701891]
 [ 1.         -0.82077735  1.15562047  1.85618152]
 ...
 [ 1.         -0.3695372  -0.09031802 -0.92485123]
 [ 1.         -0.60442933 -0.04021111 -0.84539315]
 [ 1.         -0.03397701 -0.07044252 -1.00430931]]


In [48]:
# FASE 2, probando las combinaciones posibles

def fase2(Xe, y):
    l = [0,1,2,3,4,5,6,7]
    indices = permutacion(l,5) 
    # los subconjuntos
    X = normalizacion(Xe)
    subcon = subconjuntos(indices)
    theta_Xi_y = []
    for col in subcon:
        X_i = X_1(X,col)
        theta = regresion(X_i,y) 
        theta_Xi_y.append({"X_i":X_i,"col":col,"y":y,"theta":theta})
        print(f"col:\n{col}\nX_i:\n{X_i}\ntheta:\n{theta}\n\n")
    return theta_Xi_y
    
if __name__=='__main__':
    datos = fetch_california_housing()
    X, y = datos.data,datos.target
    theta_Xi_y = fase2(X,y)
    print(f"regresion:\n{theta_Xi_y}")

col:
[4]
X_i:
[[ 1.         -0.9744286 ]
 [ 1.          0.86143887]
 [ 1.         -0.82077735]
 ...
 [ 1.         -0.3695372 ]
 [ 1.         -0.60442933]
 [ 1.         -0.03397701]]
theta:
[ 2.06855817 -0.02844396]


col:
[0]
X_i:
[[ 1.          2.34476576]
 [ 1.          2.33223796]
 [ 1.          1.7826994 ]
 ...
 [ 1.         -1.14259331]
 [ 1.         -1.05458292]
 [ 1.         -0.78012947]]
theta:
[2.06855817 0.79398939]


col:
[3]
X_i:
[[ 1.         -0.15375759]
 [ 1.         -0.26333577]
 [ 1.         -0.04901636]
 ...
 [ 1.          0.04941393]
 [ 1.          0.15877763]
 [ 1.          0.1384028 ]]
theta:
[ 2.06855817 -0.05388904]


col:
[7]
X_i:
[[ 1.         -1.32783522]
 [ 1.         -1.32284391]
 [ 1.         -1.33282653]
 ...
 [ 1.         -0.8237132 ]
 [ 1.         -0.87362627]
 [ 1.         -0.83369581]]
theta:
[ 2.06855817 -0.05304217]


col:
[2]
X_i:
[[ 1.          0.62855945]
 [ 1.          0.32704136]
 [ 1.          1.15562047]
 ...
 [ 1.         -0.09031802]
 [ 1.  

In [49]:
#METRICAS 
def metricas(X,theta,y):
    print(f"MSE, R_2, R_2_ajustado, AIC, BIC\n")
    y_ = X @ theta
    n , m = X.shape
    k = m  # predictores + intercepto
    m = m - 1 
    MSE = np.sum((y - y_)**2)/len(y)
    R_2 = 1 - np.sum((y - y_)**2)/(np.sum((y - y.mean())**2))
    R_2_ajustado = 1 - (1 - R_2)*(n - 1)/(n - m - 1)
    AIC = 2 * k + n * log(MSE)
    BIC = k * log(n) + n * log(MSE)
    #print(f"X::\n{X} \n, y::\n{y}\n, y_::\n{y_}\n")
    #print(f"theta\n{theta}")
    #print(f"errores\n{MSE} , {R_2} , {AIC} , {BIC}")
    return {"MSE":MSE,"R_2":R_2,"R_2_ajustado":R_2_ajustado,"AIC":AIC,"BIC":BIC}

if __name__=='__main__':
    datos = fetch_california_housing()
    X, y = datos.data, datos.target
    X = normalizacion(X)
    X = X_1(X,[4,2,1])
    theta = regresion(X,y)
    mtrics = metricas(X,theta,y)
    print(f"METRICAS:\n{mtrics}")

MSE, R_2, R_2_ajustado, AIC, BIC

METRICAS:
{'MSE': np.float64(1.2770737020527865), 'R_2': np.float64(0.04091215932711978), 'R_2_ajustado': np.float64(0.04077273000350967), 'AIC': 5055.951433509711, 'BIC': 5087.691378388093}


In [52]:
# FASE 3 , metricas
def fase3(data_regresion):
    # se tiene X_i   col   y   theta , cada e de data_regresion tiene esos 4 arrays
    info_final = []
    for e in data_regresion:
        X_i,col,y,theta = e["X_i"],e["col"],e["y"],e["theta"]
        mtrics = metricas(X_i,theta,y)
        info_final.append({"X_i":X_i,"col":col,"y":y,"theta":theta,"metricas":mtrics})
    return info_final

if __name__=='__main__':
    datos = fetch_california_housing()
    X, y = datos.data,datos.target
    data_regresion = fase2(X,y)
    info_final = fase3(data_regresion)
    print(f"x:\n{X} \n y:\n{y}")
    for e in info_final:
        print(f"col:\n {e["col"]}, \n {e["metricas"]}")

col:
[3]
X_i:
[[ 1.         -0.15375759]
 [ 1.         -0.26333577]
 [ 1.         -0.04901636]
 ...
 [ 1.          0.04941393]
 [ 1.          0.15877763]
 [ 1.          0.1384028 ]]
theta:
[ 2.06855817 -0.05388904]


col:
[2]
X_i:
[[ 1.          0.62855945]
 [ 1.          0.32704136]
 [ 1.          1.15562047]
 ...
 [ 1.         -0.09031802]
 [ 1.         -0.04021111]
 [ 1.         -0.07044252]]
theta:
[2.06855817 0.17533742]


col:
[1]
X_i:
[[ 1.          0.98214266]
 [ 1.         -0.60701891]
 [ 1.          1.85618152]
 ...
 [ 1.         -0.92485123]
 [ 1.         -0.84539315]
 [ 1.         -1.00430931]]
theta:
[2.06855817 0.12188183]


col:
[5]
X_i:
[[ 1.         -0.04959654]
 [ 1.         -0.09251223]
 [ 1.         -0.02584253]
 ...
 [ 1.         -0.0717345 ]
 [ 1.         -0.09122515]
 [ 1.         -0.04368215]]
theta:
[ 2.06855817 -0.02739127]


col:
[4]
X_i:
[[ 1.         -0.9744286 ]
 [ 1.          0.86143887]
 [ 1.         -0.82077735]
 ...
 [ 1.         -0.3695372 ]
 [ 1.    

In [69]:
## RESUMEN hasta la fase 3
def permutacion(l,n_sub): # l 8 columnas , n_sub 5 predictores
    f = []
    k = len(l)
    while k>1:
        i = math.floor(k*rd.uniform(0,1))
        #print(f"i : {i}")
        l[k-1] , l[i] = l[i] , l[k-1]
        f.append(l[k-1])
        if (len(l) - k) + 1  == n_sub:
            break
        k = k - 1
    #print(f)
    return f
    
def normalizacion(X):
    n , m = X.shape
    medias = np.zeros(m)
    varianzas = np.zeros(m)
    for i in range(m):
        medias[i] = X[:,i].mean()
        varianzas[i] = X[:,i].std()
    for i in range(m):
        X[:,i] = (X[:,i] - medias[i])/varianzas[i]
    return X

def X_1(Xe,columna):
    n, m = Xe.shape
    X = np.ones((n,len(columna) + 1))
    for i in range(len(columna)):
        X[:,i + 1] = Xe[:,columna[i]]
    return X

def subconjuntos(lista):
    n = len(lista)
    combinaciones = []
    # tamaños de los grupos
    for grupos in range(1, n + 1):
        # índices iniciales: [0,1,2,...,grupos-1]
        indices = list(range(grupos))
        while True:
            # construir combinación actual
            combinaciones.append([lista[i] for i in indices])
            # buscar posición para avanzar
            for i in range(grupos - 1, -1, -1):
                if indices[i] != i + n - grupos:
                    break
            else:
                # no se puede avanzar más
                break
            # avanzar este índice
            indices[i] += 1
            # reajustar los que están a la derecha
            for j in range(i + 1, grupos):
                indices[j] = indices[j - 1] + 1
    return combinaciones

def regresion(Xe,y):
    #n , m = Xe.shape
    #X = copiar_columnas(Xe,columna)  # debe ser una dependencia y no estar acoplado 
    theta = inv(Xe.T @ Xe) @ (Xe.T @ y)
    #print(f"theta:\n{theta}")
    return theta

def fase2(Xe, y):
    l = [0,1,2,3,4,5,6,7]
    indices = permutacion(l,5) 
    # los subconjuntos
    X = normalizacion(Xe)
    subcon = subconjuntos(indices)
    theta_Xi_y = []
    for col in subcon:
        X_i = X_1(X,col)
        theta = regresion(X_i,y) 
        theta_Xi_y.append({"X_i":X_i,"col":col,"y":y,"theta":theta})
        print(f"col:\n{col}\nX_i:\n{X_i}\ntheta:\n{theta}\n\n")
    return theta_Xi_y

def metricas(X,theta,y):
    #print(f"MSE, R_2, R_2_ajustado, AIC, BIC\n")
    y_ = X @ theta
    n , m = X.shape
    k = m  # predictores + intercepto
    m = m - 1 
    MSE = np.sum((y - y_)**2)/len(y)
    R_2 = 1 - np.sum((y - y_)**2)/(np.sum((y - y.mean())**2))
    R_2_ajustado = 1 - (1 - R_2)*(n - 1)/(n - m - 1)
    AIC = 2 * k + n * log(MSE)
    BIC = k * log(n) + n * log(MSE)
    #print(f"X::\n{X} \n, y::\n{y}\n, y_::\n{y_}\n")
    #print(f"theta\n{theta}")
    #print(f"errores\n{MSE} , {R_2} , {AIC} , {BIC}")
    return {"MSE":MSE,"R_2":R_2,"R_2_ajustado":R_2_ajustado,"AIC":AIC,"BIC":BIC}

def fase3(data_regresion):
    # se tiene X_i   col   y   theta , cada e de data_regresion tiene esos 4 arrays
    info_final = []
    for e in data_regresion:
        X_i,col,y,theta = e["X_i"],e["col"],e["y"],e["theta"]
        mtrics = metricas(X_i,theta,y)
        info_final.append({"X_i":X_i,"col":col,"y":y,"theta":theta,"metricas":mtrics})
    return info_final

def comparar_metricas(info_final):
    mejor_R2aj = None
    mejor_AIC = None
    mejor_BIC = None
    for e in info_final:
        col = e["col"]
        met = e["metricas"]
        if mejor_R2aj is None or met["R_2_ajustado"]>mejor_R2aj["valor"]:
            mejor_R2aj={"col":col,"valor":met["R_2_ajustado"]}
        if mejor_AIC is None or met["AIC"]<mejor_AIC["valor"]:
            mejor_AIC={"col":col,"valor":met["AIC"]}
        if mejor_BIC is None or met["BIC"]<mejor_BIC["valor"]:
            mejor_BIC = {"col":col,"valor":met["BIC"]}
    return {"mejor_R2aj":mejor_R2aj ,"mejor_AIC":mejor_AIC,"mejor_BIC":mejor_BIC}

def fase4(info_final):
    columnas = []
    R2 = []
    AIC = []
    BIC = []
    for e in info_final:
        columnas.append(e["col"])
        R2.append(e["metricas"]["R_2_ajustado"])
        AIC.append(e["metricas"]["AIC"])
        BIC.append(e["metricas"]["BIC"])
    tabla = pd.DataFrame({"columnas":columnas,"R2_ajustado":R2,"AIC":AIC,"BIC":BIC})
    tabla_final = tabla.sort_values(by="BIC",ascending=True)
    return tabla_final

if __name__=='__main__':
    datos = fetch_california_housing()
    X, y = datos.data,datos.target
    data_regresion = fase2(X,y)
    info_final = fase3(data_regresion)
    print(f"x:\n{X} \n y:\n{y}")
    for e in info_final:
        print(f"col:\n {e["col"]}, \n {e["metricas"]}")
    result = comparar_metricas(info_final) 
    print(f"\n mejor_R2aj:\n{result["mejor_R2aj"]} \n mejor_AIC:\n{result["mejor_AIC"]}\n mejor_BIC\n {result["mejor_BIC"]} ")
    print(f"tabla:\n{fase4(info_final)}")  

col:
[7]
X_i:
[[ 1.         -1.32783522]
 [ 1.         -1.32284391]
 [ 1.         -1.33282653]
 ...
 [ 1.         -0.8237132 ]
 [ 1.         -0.87362627]
 [ 1.         -0.83369581]]
theta:
[ 2.06855817 -0.05304217]


col:
[3]
X_i:
[[ 1.         -0.15375759]
 [ 1.         -0.26333577]
 [ 1.         -0.04901636]
 ...
 [ 1.          0.04941393]
 [ 1.          0.15877763]
 [ 1.          0.1384028 ]]
theta:
[ 2.06855817 -0.05388904]


col:
[0]
X_i:
[[ 1.          2.34476576]
 [ 1.          2.33223796]
 [ 1.          1.7826994 ]
 ...
 [ 1.         -1.14259331]
 [ 1.         -1.05458292]
 [ 1.         -0.78012947]]
theta:
[2.06855817 0.79398939]


col:
[5]
X_i:
[[ 1.         -0.04959654]
 [ 1.         -0.09251223]
 [ 1.         -0.02584253]
 ...
 [ 1.         -0.0717345 ]
 [ 1.         -0.09122515]
 [ 1.         -0.04368215]]
theta:
[ 2.06855817 -0.02739127]


col:
[1]
X_i:
[[ 1.          0.98214266]
 [ 1.         -0.60701891]
 [ 1.          1.85618152]
 ...
 [ 1.         -0.92485123]
 [ 1.  

In [64]:
a = (1,2)
print(f"a : {a}")
print(f"a[0] : {a[0]}  , a[1]: {a[1]}")


a : (1, 2)
a[0] : 1  , a[1]: 2


In [3]:


def fase1():
    datos = fetch_california_housing() 
    X = datos.data
    y = datos.target  
    print(f"X\n{X},shape X: {X.shape},y\n{y} shape y: {y.shape}")
    n, m = X.shape
    medias = np.zeros(m)
    varianzas = np.zeros(m)
    for i in range(m):
        medias[i] = X[:,i].mean()
        varianzas[i] = X[:,i].std()
    for i in range(m):
        X[:,i] = (X[:,i] - medias[i])/varianzas[i]
    print(f"X:\n {X}") 
    return X,y 
    
def copiar_columnas(Xe,columnas):
    n, m = Xe.shape
    X = np.ones((n,len(columnas) + 1))
    for i in range(len(columnas)):
        X[:,i + 1] = Xe[:,columnas[i]]
    return X

inv = np.linalg.inv
log = math.log

def regresion(Xe,y,columnas):
    n , m = Xe.shape
    X = copiar_columnas(Xe,columnas)
    theta = inv(X.T @ X) @ (X.T @ y)
    y_ = X @ theta
    MSE = np.sum((y - y_)**2)/len(y)
    R_2 = 1 - np.sum((y - y_)**2)/(np.sum((y - y.mean())**2))
    R_2_ajustado = 1 - (1 - R_2)*(n - 1)/(n - m - 1)
    AIC = 2 * (1+1) + n * log(MSE)
    BIC = (1+1) * log(n) + n * log(MSE)
    print(f"X::\n{X} \n, y::\n{y}\n, y_::\n{y_}\n")
    print(f"theta\n{theta}")
    print(f"errores\n{MSE} , {R_2} , {AIC} , {BIC}")
    return [MSE,R_2,R_2_ajustado,AIC,BIC],theta

def columnas(intervalo,indices):
    print(f"intervalo\n{intervalo}")
    inf = intervalo[0]
    sup = intervalo[1]
    columna = [indices[inf]]
    i = inf + 1
    while i <= sup:
        columna.append(indices[i])
        i = i + 1
    print(f"columna\n{columna}")
    return columna

def permutacion(l):
    indices = []
    k = len(l)
    while k>1: 
        i = math.floor((k*rd.uniform(0,1)))
        l[k-1], l[i] = l[i], l[k-1]
        indices.append(l[k-1])
        if len(l) - k ==5:
            break 
        k = k - 1
    return indices

def fase2(Xe, y):
    l = [0,1,2,3,4,5,6,7]
    indices = permutacion(l) 
    # los subconjuntos 
    for grupos in range(len(indices)): # grupos 
        for j in range(len(indices)): # avanza j desde inicio →  fin
            if j + grupos < len(indices):
                intervalo = (j,j + grupos) # el intervalo <j,j+grupos> 
                columna = columnas(intervalo,indices) #regresion   
                metricas = regresion(Xe,y,columna)
            
if __name__=='__main__':
    X,y = fase1()
    #X_c = copiar_columnas(X,[0,3,4])
    #print(f"X_c =\n {X_c}")
    #metricas, theta = regresion(X,y,[0,3,4])
    #columna = columnas((2,2),[0,1,2,3,4,5,6,7])
    #print(f"columna\n {columna}")
    #for i in range(5):
    #    per = permutacion([0,1,2,3,4,5,6,7])
    #   print(f"per:\n {per}")
    fase2(X,y)

X
[[   8.3252       41.            6.98412698 ...    2.55555556
    37.88       -122.23      ]
 [   8.3014       21.            6.23813708 ...    2.10984183
    37.86       -122.22      ]
 [   7.2574       52.            8.28813559 ...    2.80225989
    37.85       -122.24      ]
 ...
 [   1.7          17.            5.20554273 ...    2.3256351
    39.43       -121.22      ]
 [   1.8672       18.            5.32951289 ...    2.12320917
    39.43       -121.32      ]
 [   2.3886       16.            5.25471698 ...    2.61698113
    39.37       -121.24      ]],shape X: (20640, 8),y
[4.526 3.585 3.521 ... 0.923 0.847 0.894] shape y: (20640,)
X:
 [[ 2.34476576  0.98214266  0.62855945 ... -0.04959654  1.05254828
  -1.32783522]
 [ 2.33223796 -0.60701891  0.32704136 ... -0.09251223  1.04318455
  -1.32284391]
 [ 1.7826994   1.85618152  1.15562047 ... -0.02584253  1.03850269
  -1.33282653]
 ...
 [-1.14259331 -0.92485123 -0.09031802 ... -0.0717345   1.77823747
  -0.8237132 ]
 [-1.05458292 -0.845